## DEVLAND

Lists files that are present in each folder as well as the latest files

In [17]:
import time
import os
import glob
import pandas as pd
import subprocess
import logging
import os
import pathlib

In [18]:
directory = r'R:\RawData\Elite Star\\Devland\RAW FILES RECEIVED'

In [19]:
def get_file_list(directory):
    """Retrieve a list of all files in the directory and its subdirectories."""
    return glob.glob(os.path.join(directory, '**'), recursive=True)

In [20]:
def get_most_recent_file(directory, files):
    """Retrieve the most recent file in the directory."""
    return max(files, key=os.path.getmtime)

In [21]:
def delete_files(files):
    """Delete the specified files."""
    for file in files:
        try:
            os.remove(file)
        except Exception as e:
            logging.error(f"Error deleting file {file}: {e}")

In [22]:
def list_files(directory):
    file_paths = []

    for root, dirs, filenames in os.walk(directory):
        for filename in filenames:
            relative_path = os.path.relpath(os.path.join(root, filename), directory)
            file_paths.append(relative_path)

    df = pd.DataFrame(file_paths, columns=['File Path'])

    return df

In [23]:
def list_most_recent_files(directory):
    recent_files = []

    for root, dirs, filenames in os.walk(directory):
        if filenames:
            most_recent_file = max(filenames, key=lambda x: os.path.getmtime(os.path.join(root, x)))
            relative_path = os.path.relpath(os.path.join(root, most_recent_file), directory)
            folder_name = os.path.basename(root)
            recent_files.append([folder_name, relative_path])

    df = pd.DataFrame(recent_files, columns=['Folder Name', 'Most Recent File'])

    return df


In [24]:
output_file = r"C:\Users\tsello01\\Documents\\Data\\Devland Latest.xlsx"

try:
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        list_files(directory).to_excel(writer, sheet_name='All Files', index=False)
        list_most_recent_files(directory).to_excel(writer, sheet_name='Most Recent Files', index=False)
        
    if os.name == 'nt':  # Windows
        os.startfile(output_file)
    else:  # macOS and Linux
        subprocess.call(['open', output_file])

except PermissionError:
    print("Error: The file is already open. Please close it and try again.")